In [1]:
import requests
from sqlalchemy.orm import sessionmaker
from settings import Engine
from models import Material, OpticalIndex

ModuleNotFoundError: No module named 'settings'

## filmetricsからデータを追加

In [2]:
def add_data(url):
    response = requests.get(url)
    if response.status_code == 200:
        
        name = url.split('/')[-1]
        text = response.text
        
        Session = sessionmaker(bind=Engine)
        session = Session()

        material = session.query(Material).filter_by(name=name).one_or_none()
        if material is None:
            material = Material(name=name)
            session.add(material)
            material = session.query(Material).filter_by(name=name).one()
            lines = text.splitlines()
            lines.pop(0)
            opticalindexes = [OpticalIndex(material_id=material.id,wavelength=line.split()[0],
                                           n=line.split()[1],k=line.split()[2]) for line in lines]
            session.add_all(opticalindexes)
            session.commit()
        session.close()
    return response.status_code



## データ出力

In [3]:
def get_data(name):
    Session = sessionmaker(bind=Engine)
    session = Session()
    opticalindexes = session.query(OpticalIndex).\
                             join(Material,OpticalIndex.material_id == Material.id).\
                             filter_by(name=name).order_by(OpticalIndex.wavelength).all()
    result = [(opticalindex.wavelength, complex(opticalindex.n,opticalindex.k)) for opticalindex in opticalindexes]
    session.close()
    return result

In [4]:
def get_material_name():
    Session = sessionmaker(bind=Engine)
    session = Session()
    qs = session.query(Material).all()
    session.close()
    return [q.name for q in qs]

## データ削除

In [5]:
def delete_data(name):
    Session = sessionmaker(bind=Engine)
    session = Session()
    q = session.query(Material).filter_by(name=name).one_or_none()
    if q is not None:
        session.query(OpticalIndex).filter_by(material_id=q.id).delete()
        session.delete(q)
        session.commit()
    session.close()


In [6]:
# Test
url = 'https://www.filmetricsinc.jp/technology/refractive-index-database/download/Acrylic'
add_data(url)


200

In [7]:
names= get_material_name()
print(get_data(names[0]))
delete_data(names[0])

[(435.0, (1.5027+0j)), (440.0, (1.5017+0j)), (445.0, (1.5009+0j)), (450.0, (1.5002+0j)), (455.0, (1.4996+0j)), (460.0, (1.4991+0j)), (465.0, (1.4987+0j)), (470.0, (1.4983+0j)), (475.0, (1.498+0j)), (480.0, (1.4977+0j)), (485.0, (1.4974+0j)), (490.0, (1.4971+0j)), (495.0, (1.4968+0j)), (500.0, (1.4965+0j)), (505.0, (1.4962+0j)), (510.0, (1.496+0j)), (515.0, (1.4957+0j)), (520.0, (1.4954+0j)), (525.0, (1.4951+0j)), (530.0, (1.4948+0j)), (535.0, (1.4945+0j)), (540.0, (1.4942+0j)), (545.0, (1.4939+0j)), (550.0, (1.4936+0j)), (555.0, (1.4933+0j)), (560.0, (1.493+0j)), (565.0, (1.4927+0j)), (570.0, (1.4924+0j)), (575.0, (1.4921+0j)), (580.0, (1.4918+0j)), (585.0, (1.4915+0j)), (590.0, (1.4913+0j)), (595.0, (1.491+0j)), (600.0, (1.4907+0j)), (605.0, (1.4904+0j)), (610.0, (1.4902+0j)), (615.0, (1.4899+0j)), (620.0, (1.4896+0j)), (625.0, (1.4894+0j)), (630.0, (1.4891+0j)), (635.0, (1.4889+0j)), (640.0, (1.4887+0j)), (645.0, (1.4884+0j)), (650.0, (1.4882+0j)), (655.0, (1.488+0j)), (660.0, (1.487

In [10]:
import database as db


In [11]:
db.add_data('https://www.filmetricsinc.jp/technology/refractive-index-database/download/Al')

200

In [12]:
db.get_data('Al')

[(183.68, (0.102+2.07j)),
 (185.05, (0.1044+2.085002j)),
 (186.44, (0.1062+2.103344j)),
 (187.86, (0.1076+2.124184j)),
 (189.29, (0.1088+2.146683j)),
 (190.75, (0.11+2.17j)),
 (192.23, (0.1113+2.193295j)),
 (193.73, (0.1129+2.215727j)),
 (195.25, (0.1148+2.237511j)),
 (196.8, (0.1168+2.258863j)),
 (198.38, (0.119+2.28j)),
 (199.98, (0.1213+2.301136j)),
 (201.6, (0.1236+2.322489j)),
 (203.25, (0.1258+2.344273j)),
 (204.93, (0.128+2.366705j)),
 (206.64, (0.13+2.39j)),
 (208.38, (0.1318+2.414375j)),
 (210.14, (0.134+2.44j)),
 (211.94, (0.1364+2.465625j)),
 (213.77, (0.139+2.49j)),
 (215.63, (0.1415+2.509375j)),
 (217.52, (0.144+2.53j)),
 (219.44, (0.1464+2.559375j)),
 (221.4, (0.149+2.59j)),
 (223.4, (0.1519+2.615j)),
 (225.43, (0.155+2.64j)),
 (227.5, (0.1585+2.669375j)),
 (229.6, (0.162+2.7j)),
 (231.75, (0.1649+2.73j)),
 (233.93, (0.168+2.76j)),
 (236.16, (0.1719+2.79j)),
 (238.43, (0.176+2.82j)),
 (240.75, (0.1796+2.85j)),
 (243.11, (0.183+2.88j)),
 (245.52, (0.1865+2.909375j)),
 (247